**Demo for `teneva.core.anova`**

---

This module contains the function "anova" which computes the TT-approximation for the tensor by TT-ANOVA method, using given random samples.

## Loading and importing modules

In [1]:
import numpy as np
import teneva
from time import perf_counter as tpc
np.random.seed(42)

## Function `anova`

Construct TT-ANOVA from the given random tensor samples.

In [2]:
d         = 5                           # Dimension of the function
a         = [-5., -4., -3., -2., -1.]   # Lower bounds for spatial grid
b         = [+6., +3., +3., +1., +2.]   # Upper bounds for spatial grid
n         = [ 20,  18,  16,  14,  12]   # Shape of the tensor

In [3]:
m         = 1.E+4  # Number of calls to target function
order     = 1      # Order of ANOVA decomposition (1 or 2)
r         = 2      # TT-rank of the resulting tensor

We set the target function (the function takes as input a set of tensor multi-indices I of the shape [samples, dimension], which are transformed into points X of a uniform spatial grid using the function "ind_to_poi"):

In [4]:
from scipy.optimize import rosen
def func(I): 
    X = teneva.ind_to_poi(I, a, b, n)
    return rosen(X.T)

We prepare train data from the LHS random distribution:

In [5]:
I_trn = teneva.sample_lhs(n, m) 
Y_trn = func(I_trn)

We prepare test data from random tensor multi-indices:

In [6]:
# Number of test points:
m_tst = int(1.E+4)

# Random multi-indices for the test points:
I_tst = np.vstack([np.random.choice(k, m_tst) for k in n]).T

# Function values for the test points:
Y_tst = func(I_tst)

We build the TT-tensor, which approximates the target function:

In [7]:
t = tpc()
Y = teneva.anova(I_trn, Y_trn, r, order)
t = tpc() - t

print(f'Build time     : {t:-10.2f}')

Build time     :       0.01


And now we can check the result:

In [8]:
# Fast getter for TT-tensor values:
get = teneva.getter(Y)                     

# Compute approximation in train points:
Z = np.array([get(i) for i in I_trn])

# Accuracy of the result for train points:
e_trn = np.linalg.norm(Z - Y_trn)          
e_trn /= np.linalg.norm(Y_trn)

# Compute approximation in test points:
Z = np.array([get(i) for i in I_tst])

# Accuracy of the result for test points:
e_tst = np.linalg.norm(Z - Y_tst)          
e_tst /= np.linalg.norm(Y_tst)

print(f'Error on train : {e_trn:-10.2e}')
print(f'Error on test  : {e_tst:-10.2e}')

Error on train :   1.11e-01
Error on test  :   1.14e-01


We can also build approximation using 2-th order ANOVA decomposition:

In [9]:
t = tpc()
Y = teneva.anova(I_trn, Y_trn, r, order=2)
t = tpc() - t

get = teneva.getter(Y)                     

Z = np.array([get(i) for i in I_trn])
e_trn = np.linalg.norm(Z - Y_trn)          
e_trn /= np.linalg.norm(Y_trn)

Z = np.array([get(i) for i in I_tst])
e_tst = np.linalg.norm(Z - Y_tst)          
e_tst /= np.linalg.norm(Y_tst)

print(f'Build time     : {t:-10.2f}')
print(f'Error on train : {e_trn:-10.2e}')
print(f'Error on test  : {e_tst:-10.2e}')

Build time     :       0.15
Error on train :   8.67e-02
Error on test  :   8.18e-02


Let's look at the quality of approximation for a linear function:

In [10]:
d = 4
a = -2.
b = +3.
n = [10] * d
r = 3
m_trn = int(1.E+5)
m_tst = int(1.E+4)

In [11]:
def func(I): 
    X = teneva.ind_to_poi(I, a, b, n)
    return 5. + 0.1 * X[:, 0] + 0.2 * X[:, 1] + 0.3 * X[:, 2] + 0.4 * X[:, 3]

In [12]:
I_trn = teneva.sample_lhs(n, m_trn) 
Y_trn = func(I_trn)

I_tst = np.vstack([np.random.choice(n[i], m_tst) for i in range(d)]).T
Y_tst = func(I_tst)

In [13]:
t = tpc()
Y = teneva.anova(I_trn, Y_trn, r, order=1)
t = tpc() - t

get = teneva.getter(Y)                     

Z = np.array([get(i) for i in I_trn])
e_trn = np.linalg.norm(Z - Y_trn)          
e_trn /= np.linalg.norm(Y_trn)

Z = np.array([get(i) for i in I_tst])
e_tst = np.linalg.norm(Z - Y_tst)          
e_tst /= np.linalg.norm(Y_tst)

print(f'Build time     : {t:-10.2f}')
print(f'Error on train : {e_trn:-10.2e}')
print(f'Error on test  : {e_tst:-10.2e}')

Build time     :       0.03
Error on train :   2.50e-03
Error on test  :   2.49e-03


Let's look at the quality of approximation for a quadratic function

In [14]:
d = 4
a = -2.
b = +3.
n = [10] * d
r = 3
m_trn = int(1.E+5)
m_tst = int(1.E+4)

In [15]:
def func(I): 
    X = teneva.ind_to_poi(I, a, b, n)
    return 5. + 0.1 * X[:, 0]**2 + 0.2 * X[:, 1]**2 + 0.3 * X[:, 2]**2 + 0.4 * X[:, 3]**2

In [16]:
I_trn = teneva.sample_lhs(n, m_trn) 
Y_trn = func(I_trn)

I_tst = np.vstack([np.random.choice(n[i], m_tst) for i in range(d)]).T
Y_tst = func(I_tst)

In [17]:
t = tpc()
Y = teneva.anova(I_trn, Y_trn, r, order=1)
t = tpc() - t

get = teneva.getter(Y)                     

Z = np.array([get(i) for i in I_trn])
e_trn = np.linalg.norm(Z - Y_trn)          
e_trn /= np.linalg.norm(Y_trn)

Z = np.array([get(i) for i in I_tst])
e_tst = np.linalg.norm(Z - Y_tst)          
e_tst /= np.linalg.norm(Y_tst)

print(f'Build time     : {t:-10.2f}')
print(f'Error on train : {e_trn:-10.2e}')
print(f'Error on test  : {e_tst:-10.2e}')

Build time     :       0.03
Error on train :   2.60e-03
Error on test  :   2.59e-03


---